Importing Python packages

In [4]:
import pandas as pd
from Utils.preprocessing import *

Importing datasets

In [5]:
path = './Datasets/Training/Small'
file_names = get_file_names(folder_path=path,returns=False)
file_names[0]

File names:  ['Sample02.csv', '._.DS_Store', 'Sample01.csv', 'Sample06.csv', 'Sample05.csv', 'Sample09.csv', 'Sample11.csv', 'Sample07.csv', 'Sample10.csv', 'Sample03.csv', 'Sample12.csv', 'Sample04.csv', 'Sample08.csv']
Number of files:  13


'Sample02.csv'

In [6]:
df_path = path + '/' + file_names[0]
df = pd.read_csv(df_path)
df

,Time(s),Flow_Rate(ml/m),Upstream_Pressure(psi),Downstream_Pressure(psi)
0,0.0,9.836354,1.28750,0.55000
1,0.1,9.935067,0.50625,0.51250
2,0.2,11.317054,-0.10000,-1.02500
3,0.3,7.565948,1.05625,0.21250
4,0.4,12.205473,1.10625,0.77500
...,...,...,...,...
3265,326.5,266.885785,10.43750,-0.85000
3266,326.6,267.280639,11.33750,0.42500
3267,326.7,266.984499,9.73750,0.13125
3268,326.8,259.482288,10.01875,-0.76250
